In [1]:
from llamp.mp.agent import MPLLM

mpllm = MPLLM()

print("MPLLM is ready to serve you!")

# user_input = input("Enter materials-related question: ")
# user_input = "What is the bandgap of stable BaTiO3?"
user_input = "What is the elastic tensor of NaCl? Summarize it in text for me."
user_input = "What is the bandgap of stable MgSe?"
# user_input = "Is YbCl3 magnetic or non-magnetic?"
# # user_input = "What is the energy above hull of Cu3P4Pb3O16 at 0K?"
# # user_input = "Is Cu stable in FCC or BCC structure at 0K based on the energy distance above hull?"
# # user_input = "Do we have stable Cu-substituted lead phosphate apatite at 0K?"
# # user_input = "What is the bandgap of the most stable magnetite?"
# user_input = "Search for the necessary information for FCC Cu and summarize for me."
user_input = "Pull out the crystal structure of BCC Al using summary endpoint and print out pymatgen structure dictionary for me"
# # user_input = "What is the crystal structure of LiFePO4?"
# # user_input = "Give me 10 entries (including chemical formula and material id) closest to the GGA energy hull in the Mg-Ni-Zn system at 0K"
# user_input = "What are the oxidation states of atoms in magnetite? Feed relevant fields to request MP API and print out the results for me."
# user_input = "Give me some example recipes to synthesize Li4Ti5O12"

mpllm.run_material_conversation(
    user_input=user_input, 
    # model='gpt-4-32k-0314', 
    model='gpt-3.5-turbo-16k-0613', 
    debug=True
    )
# mpllm.run_material_conversation(user_input, model="gpt-3.5-turbo-16k-0613", debug=True)

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


MPLLM is ready to serve you!
Conversation started.
User: Pull out the crystal structure of BCC Al using summary endpoint and print out pymatgen structure dictionary for me
OpenAI General: {
  "id": "chatcmpl-86C9P8KuBY8pAg5iqKchkWT3GZPII",
  "object": "chat.completion",
  "created": 1696487367,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Call MP"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 149,
    "completion_tokens": 2,
    "total_tokens": 151
  }
}
LLaMP Function Calling: {
  "id": "chatcmpl-86C9R5ncUEROVLN2cnDlVDJxsUrXg",
  "object": "chat.completion",
  "created": 1696487369,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "search_materials_summary__get",
          "arguments": "{\n  \"formula\": \"A

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

MP API response: [{"structure": {"@module": "pymatgen.core.structure", "@class": "Structure", "charge": 0, "lattice": {"matrix": [[1.59121869, 1.59121869, 1.59121869], [1.59121869, -1.59121869, -1.59121869], [-1.59121869, 1.59121869, -1.59121869]], "pbc": [true, true, true], "a": 2.756071617033191, "b": 2.756071617033191, "c": 2.756071617033191, "alpha": 109.47122063449069, "beta": 109.47122063449069, "gamma": 109.47122063449069, "volume": 16.11571598722545}, "sites": [{"species": [{"element": "Al", "occu": 1}], "abc": [0.0, -0.0, 0.0], "xyz": [0.0, 0.0, 0.0], "label": "Al", "properties": {"magmom": 0.0}}]}}]
OpenAI Text Completion: {
  "id": "chatcmpl-86C9VhmjdX3LeBhRpAOVC9KXWFl4q",
  "object": "chat.completion",
  "created": 1696487373,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The material is a single crystal structure of Aluminum (Al) with a lattice parameter of 2.756 \u00c5 and a 

KeyboardInterrupt: Interrupted by user

In [5]:
print(f"""The third method involves solid-state synthesis to prepare Li4Ti5O12. Here is a summary of the procedures and the operation conditions for each step:\n\n1. Precursors: Li2CO3 and titanium dioxide (TiO2).\n\n2. Mixing: Thoroughly mix the Li2CO3 and TiO2 powders in the desired stoichiometric ratio. This can be done using a mortar and pestle or a ball mill.\n\n3. Grinding: Grind the mixture to ensure homogeneity and to enhance the reaction kinetics. This step helps in achieving a uniform distribution of the reactants.\n\n4. Heating: Transfer the ground mixture to a crucible or a ceramic boat and heat it in a furnace. The heating temperature and duration depend on the specific synthesis conditions mentioned in the literature. Typically, the mixture is heated at temperatures ranging from 700\u00b0C to 900\u00b0C for several hours.\n\n5. Cooling: After the desired heating duration, allow the sample to cool down slowly to room temperature. This slow cooling process helps in minimizing the formation of unwanted phases and ensures the stability of the final Li4Ti5O12 product.\n\n6. Characterization: Once the sample has cooled down, it can be characterized using various techniques such as X-ray diffraction (XRD), scanning electron microscopy (SEM), and energy-dispersive X-ray spectroscopy (EDS) to confirm the formation of Li4Ti5O12 and to analyze its structural and morphological properties.\n\nIt is important to note that the specific operation conditions, such as the heating temperature, duration, and cooling rate, may vary depending on the specific synthesis conditions mentioned in the literature source. Therefore, it is recommended to refer to the original research paper for detailed information on the exact operation conditions used in the synthesis of Li4Ti5O12.""")

The third method involves solid-state synthesis to prepare Li4Ti5O12. Here is a summary of the procedures and the operation conditions for each step:

1. Precursors: Li2CO3 and titanium dioxide (TiO2).

2. Mixing: Thoroughly mix the Li2CO3 and TiO2 powders in the desired stoichiometric ratio. This can be done using a mortar and pestle or a ball mill.

3. Grinding: Grind the mixture to ensure homogeneity and to enhance the reaction kinetics. This step helps in achieving a uniform distribution of the reactants.

4. Heating: Transfer the ground mixture to a crucible or a ceramic boat and heat it in a furnace. The heating temperature and duration depend on the specific synthesis conditions mentioned in the literature. Typically, the mixture is heated at temperatures ranging from 700°C to 900°C for several hours.

5. Cooling: After the desired heating duration, allow the sample to cool down slowly to room temperature. This slow cooling process helps in minimizing the formation of unwanted p

In [3]:
print(f"""To start with the first method, solid-state synthesis using TiO2 (anatase) and LiOH\u00b7H2O as precursors, you will need to follow these steps:\n\n1. Measure and weigh the required amounts of TiO2 (anatase) and LiOH\u00b7H2O according to the stoichiometric ratio of the reaction: 4 LiOH\u00b7H2O + 5 TiO2 -> 1 Li4Ti5O12 + 6 H2O.\n\n2. Thoroughly mix the weighed precursors in a mortar and pestle or a ball mill to ensure homogeneity.\n\n3. Transfer the mixture to a crucible or ceramic boat.\n\n4. Place the crucible or ceramic boat in a furnace and heat it under an oxygen atmosphere.\n\n5. Ramp the temperature gradually to the desired annealing temperature, typically around 700-900\u00b0C, and hold it for a specific duration, usually several hours.\n\n6. After the annealing process, allow the sample to cool down slowly to room temperature.\n\n7. Remove the sample from the furnace and store it in a desiccator to prevent moisture absorption.\n\n8. Characterize the synthesized Li4Ti5O12 sample using techniques such as X-ray diffraction (XRD), scanning electron microscopy (SEM), and energy-dispersive X-ray spectroscopy (EDS) to confirm the formation of the desired phase.\n\nIt is important to note that the specific annealing temperature and duration may vary depending on the equipment and conditions used. Therefore, it is recommended to refer to the original research paper or consult with experts in the field for precise synthesis parameters.""")

To start with the first method, solid-state synthesis using TiO2 (anatase) and LiOH·H2O as precursors, you will need to follow these steps:

1. Measure and weigh the required amounts of TiO2 (anatase) and LiOH·H2O according to the stoichiometric ratio of the reaction: 4 LiOH·H2O + 5 TiO2 -> 1 Li4Ti5O12 + 6 H2O.

2. Thoroughly mix the weighed precursors in a mortar and pestle or a ball mill to ensure homogeneity.

3. Transfer the mixture to a crucible or ceramic boat.

4. Place the crucible or ceramic boat in a furnace and heat it under an oxygen atmosphere.

5. Ramp the temperature gradually to the desired annealing temperature, typically around 700-900°C, and hold it for a specific duration, usually several hours.

6. After the annealing process, allow the sample to cool down slowly to room temperature.

7. Remove the sample from the furnace and store it in a desiccator to prevent moisture absorption.

8. Characterize the synthesized Li4Ti5O12 sample using techniques such as X-ray dif